In [1]:
import time
from collections import namedtuple
import numpy as np
import tensorflow as tf

The history saving thread hit an unexpected error (DatabaseError('database disk image is malformed',)).History will not be written to the database.


In [2]:
with open('tanshi.txt','r') as of:
    text=of.read()

vocab = set(text)
vocab_to_int = {c:i for i,c in enumerate(vocab)}

int_to_vocab = dict(enumerate(vocab))
print(int_to_vocab[ 1000])
print(vocab_to_int['落'])
print (vocab_to_int['，']) #换行符号也作为一个文字存在
encoded = np.array([vocab_to_int[c] for c in text], dtype=np.int32)
print ('转换后的数字化文本 \n',encoded)


灞
879
261
转换后的数字化文本 
 [ 521  808 1042 ...,  496  173  921]


In [5]:
#检查前1000个编码
encoded[:100]

array([ 521,  808, 1042,  277,  598,  261,  135,  696,  339,  822,  677,
        921, 1304, 1204,  132,  356,  549,  860,  261, 1019,  879,  903,
        309, 1285,  921, 1304, 1251,   35,  175, 1114,  647,  261,  646,
        157,  965,  819, 1075,  921, 1304,  730,  146,  182,  415,  431,
        261, 1228,  380, 1289,  201,  273,  921, 1304, 1226, 1305,  503,
        719,  793,  261,  680,  458,  227,  164,  983,  921, 1304,  455,
       1220,  823,  465,  385,  261,  655,  598, 1246, 1149, 1134,  921,
       1304, 1265,  780,  341, 1011, 1225,  261,  800,  985, 1228,  702,
        231,  921, 1304,  922,  630,  655,  598, 1219,  261,  434,  137,
       1152])

In [6]:
#文字的unique个数
len(vocab)

1334

In [7]:
#train data deal wtih
import numpy as np

arr=range(1,601)

arr=np.array(arr)

n_seqs=3

n_steps=20

characters_per_batch = n_seqs*n_steps  #60

n_batches = len(arr)//characters_per_batch #10批

arr = arr[:n_batches * characters_per_batch] #1:600


arr = arr.reshape(n_seqs,-1)

print('转换后的形状',arr.shape)

i=0

for n in range(0,arr.shape[1],n_steps): 
    i=i+1
    print('第',i,'次切，一共会切分batch数量那么多次,本例为10次')
    
    x = arr[:, n:n+n_steps]
    print('x 当前为 \n',x)                      # 切出的x
    print('x[:,1:] 当前为 \n' ,x[:,1:])         #切出的x除去第一列的后面4列，作为y的前面4列
    print('x[:,0] 当前为' ,x[:,0])              #x的第一列，将来作为y的最后一列
    
    
    # 对应label输出, shift 一次。也就是知道x=1234，设置y=2341
    y = np.zeros_like(x)
    y[:, :-1], y[:, -1] = x[:, 1:], x[:, 0]
    print('y 转换后为 \n',y)
    





转换后的形状 (3, 200)
第 1 次切，一共会切分batch数量那么多次,本例为10次
x 当前为 
 [[  1   2   3   4   5   6   7   8   9  10  11  12  13  14  15  16  17  18
   19  20]
 [201 202 203 204 205 206 207 208 209 210 211 212 213 214 215 216 217 218
  219 220]
 [401 402 403 404 405 406 407 408 409 410 411 412 413 414 415 416 417 418
  419 420]]
x[:,1:] 当前为 
 [[  2   3   4   5   6   7   8   9  10  11  12  13  14  15  16  17  18  19
   20]
 [202 203 204 205 206 207 208 209 210 211 212 213 214 215 216 217 218 219
  220]
 [402 403 404 405 406 407 408 409 410 411 412 413 414 415 416 417 418 419
  420]]
x[:,0] 当前为 [  1 201 401]
y 转换后为 
 [[  2   3   4   5   6   7   8   9  10  11  12  13  14  15  16  17  18  19
   20   1]
 [202 203 204 205 206 207 208 209 210 211 212 213 214 215 216 217 218 219
  220 201]
 [402 403 404 405 406 407 408 409 410 411 412 413 414 415 416 417 418 419
  420 401]]
第 2 次切，一共会切分batch数量那么多次,本例为10次
x 当前为 
 [[ 21  22  23  24  25  26  27  28  29  30  31  32  33  34  35  36  37  38
   39  40]
 [221 222 223 224

In [8]:
#利用yield 数据生成器 constructor,供应数据
def get_batches(arr,n_seqs,n_steps):
    
    characters_per_batch = n_seqs * n_steps
    n_batches = len(arr)//characters_per_batch
    
    arr = arr[:n_batches * characters_per_batch]
    
    arr = arr.reshape((n_seqs,-1))
    
    for n in range(0,arr.shape[1],n_steps):
        
        x = arr[:, n:n+n_steps]
        
        y = np.zeros_like(x)
        
        y[:, :-1], y[:, -1] = x[:, 1:], x[:,0]
        
        yield x ,y

In [13]:
batches = get_batches(encoded,3,20)

#get_barches 函数中的
x, y = next(batches)

In [14]:
print('x \n ',x)
print('y \n ',y)


x 
  [[ 521  808 1042  277  598  261  135  696  339  822  677  921 1304 1204
   132  356  549  860  261 1019]
 [1038  279   81  261  493 1133  473  349  146  921 1304  612   87  415
  1245 1090  261    0  201  647]
 [ 178  879  905  197   57  261 1283  766  774  915  449  921 1304   42
  1128  371  428  339  261 1180]]
y 
  [[ 808 1042  277  598  261  135  696  339  822  677  921 1304 1204  132
   356  549  860  261 1019  521]
 [ 279   81  261  493 1133  473  349  146  921 1304  612   87  415 1245
  1090  261    0  201  647 1038]
 [ 879  905  197   57  261 1283  766  774  915  449  921 1304   42 1128
   371  428  339  261 1180  178]]


In [17]:
def build_inputs(batch_size, num_steps):
    inputs = tf.placeholder(tf.int32, [batch_size, num_steps], name='inputs')
    targets = tf.placeholder(tf.int32, [batch_size,num_steps], name='targets')
    
    keep_prob = tf.placeholder(tf.float32, name='keep_prob')
    
    print('build_inputs 函数中的input',input)
    return inputs, targets, keep_prob

In [18]:
def build_lstm(lstm_size,num_layers, batch_size,keep_prob):
    
    lstm = tf.contrib.rnn.BasicLSTMCell(lstm_size)
    
    drop = tf.contrib.rnn.DropoutWrapper(lstm, output_keep_prob=keep_prob)
    
    cell = tf.contrib.rnn.MultiRNNCell( [drop] * num_layers )
    
    initial_state = cell.zero_state(batch_size, tf.float32)
    
    print('build LSTM函数中传入的batch_size',batch_size)
    
    return cell, initial_state


In [27]:
def build_output(lstm_output, in_size, out_size):
    
    seq_output = tf.concat(lstm_output, axis=1)
    print('build_output 函数中的seq_output 是否为10列 60行 ：',seq_output) #需要删除
    x = tf.reshape(seq_output,[-1, in_size])
    print('build_output 函数中的 reshape seqput ', x) #需要删除
    
    # Connect the RNN outputs to a softmax layer
    #连接RNN的结果到 softmax layer 
    with tf.variable_scope('softmax'):
        softmax_w = tf.Variable(tf.truncated_normal((in_size,out_size),stddev=0.1))
        softmax_b = tf.Variable(tf.zeros(out_size))
        
        #logits应该是 (M * N ) * (字典个数列)
    logits = tf.matmul(x,softmax_w) + softmax_b
    print('build_output 函数中的logit 是否是字典个数:',logits)
    # Use softmax to get the probabilities for predicted charactersprint('b')
        #输出预测
    out = tf.nn.softmax(logits, name='predictions')
    print('build_output 函数中的softmax_out',out)
    return out, logits
        
    

In [20]:
def build_loss(logits, targets, lstm_size, num_classes):
    
    y_one_hot = tf.one_hot(targets, num_classes)
    y_reshaped = tf.reshape(y_one_hot, logits.get_shape())
    
     # Softmax cross entropy loss
    #获得loss 以交叉熵作为损失
    loss = tf.nn.softmax_cross_entropy_with_logits(logits=logits, labels=y_reshaped)
    loss = tf.reduce_mean(loss)
    return loss

In [29]:
def build_optimizer(loss, learning_rate, grad_clip):
    
    tvars = tf.trainable_variables()
    
    grads, _ = tf.clip_by_global_norm(tf.gradients(loss,tvars), grad_clip)
    
    #设置采用AdamOptimizer优化器
    train_op = tf.train.AdadeltaOptimizer(learning_rate)
    
    optimizer = train_op.apply_gradients(zip(grads,tvars))
    
    return optimizer

    

In [22]:
class CharRNN:
    #If before you were using: MultiRNNCell([BasicLSTMCell(...)] * num_layers), 
    #change to: MultiRNNCell([BasicLSTMCell(...) for _ in range(num_layers)]). 
    #num_classes对应前面输出结构的大小。我们这里是字典的数量245
    def __init__(self, num_classes, batch_size=3, num_steps=20, 
                       lstm_size=10, num_layers=5, learning_rate=0.01, 
                       grad_clip=5, sampling=False):
        '''
        参数说明及默认设置说明：
        num_classes 为分类，如果是情感分析就是2类。我们这里由于需要输出文章，相对更复杂一些，类似对文章的自编码。
        batch_size=4 ，定义单个batch 4个序列
        num_steps=20,定义一个序列默认20个字符
        lstm_size=5 ，定义单个LSTM Cell 中有5个unit。
        num_layers=5 ，定义LSTM Cell 有5层
        learning_rate=0.01，定义学习率，注意0.001这样更细的学习率可能更好，此处设置为0.01主要是为了个人电脑CPU跟快收敛一些。
        sampling=Fasle帮助我们做一些基础采样学习，或者创建基准用
        sampling==True 表示 batch_size, num_steps = 1, 1 ,sampling==False 设置为batch_size, num_steps = batch_size, num_steps
        
        '''
        
    
        # 当采样Sampling == True, 一次处理一个字符

        if sampling == True:
            batch_size, num_steps = 1, 1
        else:
            batch_size, num_steps = batch_size, num_steps
        
        #复位default graph
        tf.reset_default_graph()
        
        # 通过build_inputs返回 3个 placeholder tensors
        self.inputs, self.targets, self.keep_prob = build_inputs(batch_size, num_steps)

        # 创建LSTM cell 返回LSTM Cell和初始状态
        cell, self.initial_state = build_lstm(lstm_size, num_layers, batch_size, self.keep_prob)

        ### 处理数据
        #  对输入做 one-hot encode 
        x_one_hot = tf.one_hot(self.inputs, num_classes)
       
        
        # Run each sequence step through the RNN and collect the outputs
        #运行每一个序列step，
        #传入1 lstm cell对象以及 2 输入数据x（一个batch中的4个序列） 和 3 LSTM最后一层的cell states 初始状态
        #返回输出和state
        print(x_one_hot)
        outputs, state = tf.nn.dynamic_rnn(cell, x_one_hot, initial_state=self.initial_state)
        self.final_state = state
        
        # 获得 softmax predictions 和 logits
        self.prediction, self.logits = build_output(outputs, lstm_size, num_classes)
        
        # 定义损失函数 和带梯度修剪的梯度下降优化器 
        self.loss = build_loss(self.logits, self.targets, lstm_size, num_classes)
        self.optimizer = build_optimizer(self.loss, learning_rate, grad_clip)

In [23]:
#设置训练超参数
batch_size = 3       
num_steps = 20        
lstm_size = 10        
num_layers = 5         
learning_rate = 0.01   
keep_prob = 0.5   

In [30]:
epochs = 20
# 存放变量20，用于后来的保存
save_every_n = 20

model = CharRNN(len(vocab), batch_size=batch_size, num_steps=num_steps,
                lstm_size=lstm_size, num_layers=num_layers, 
                learning_rate=learning_rate)

#只保留最近5个checkpoint
saver = tf.train.Saver(max_to_keep=5)

with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    
    # 如果加入下面的注释 可以继续训练
    #saver.restore(sess, 'checkpoints/______.ckpt')
    counter = 0
    #new_state = sess.run(model.initial_state)
    for e in range(epochs):
        # 训练网络
        #设置第一次的state是模型的初始state，都是0
        new_state = sess.run(model.initial_state)
        #print(new_state)
        loss = 0
        for x, y in get_batches(encoded, batch_size, num_steps):
            #单次get_batches会获得4个序列的x 和 y
            #计数器每次加一
            counter += 1
            #print('new state count',counter,new_state)
            #记录时间
            start = time.time()
            #创建feed字典
            feed = {model.inputs: x,
                    model.targets: y,
                    model.keep_prob: keep_prob,
                    model.initial_state: new_state}
            #运行会话model.loss,model.final_state,model.optimizer
            batch_loss, new_state, _ = sess.run([model.loss, 
                                                 model.final_state, 
                                                 model.optimizer], 
                                                 feed_dict=feed)
            
            #记录单批运算时间
            end = time.time()
            #打印当前Epoch,当前训练batch对应的step，当前batch对应的loss，当前step对应的消耗
            print('Epoch: {}/{}... '.format(e+1, epochs),
                  'Training Step: {}... '.format(counter),
                  'Training loss: {:.4f}... '.format(batch_loss),
                  '{:.4f} sec/batch'.format((end-start)))
            
            #每20次训练保存一次，保存到checkpoints文件夹文件名为i{计数器}_l{lstm_unit个数}
            if (counter % save_every_n == 0):
                saver.save(sess, "checkpoints/i{}_l{}.ckpt".format(counter, lstm_size))
    
    saver.save(sess, "checkpoints/i{}_l{}.ckpt".format(counter, lstm_size))

build_inputs 函数中的input <bound method Kernel.raw_input of <ipykernel.ipkernel.IPythonKernel object at 0x0000000004BB3860>>
build LSTM函数中传入的batch_size 3
Tensor("one_hot:0", shape=(3, 20, 1334), dtype=float32)
build_output 函数中的seq_output 是否为10列 60行 ： Tensor("concat:0", shape=(3, 20, 10), dtype=float32)
build_output 函数中的 reshape seqput  Tensor("Reshape:0", shape=(60, 10), dtype=float32)
build_output 函数中的logit 是否是字典个数: Tensor("add:0", shape=(60, 1334), dtype=float32)
build_output 函数中的softmax_out Tensor("predictions:0", shape=(60, 1334), dtype=float32)
Epoch: 1/20...  Training Step: 1...  Training loss: 7.1960...  0.1890 sec/batch
Epoch: 1/20...  Training Step: 2...  Training loss: 7.1962...  0.0430 sec/batch
Epoch: 1/20...  Training Step: 3...  Training loss: 7.1960...  0.0360 sec/batch
Epoch: 1/20...  Training Step: 4...  Training loss: 7.1959...  0.0410 sec/batch
Epoch: 1/20...  Training Step: 5...  Training loss: 7.1959...  0.0370 sec/batch
Epoch: 1/20...  Training Step: 6...  Training l

In [31]:
new_state[0][0].shape

(3, 10)

In [32]:
tf.train.get_checkpoint_state('checkpoints')

model_checkpoint_path: "checkpoints\\i2340_l10.ckpt"
all_model_checkpoint_paths: "checkpoints\\i2260_l10.ckpt"
all_model_checkpoint_paths: "checkpoints\\i2280_l10.ckpt"
all_model_checkpoint_paths: "checkpoints\\i2300_l10.ckpt"
all_model_checkpoint_paths: "checkpoints\\i2320_l10.ckpt"
all_model_checkpoint_paths: "checkpoints\\i2340_l10.ckpt"

In [33]:
def pick_top_n(preds, vocab_size, top_n=500):
    
    p = np.squeeze(preds)
    p[np.argsort(p)[:-top_n]] = 0
    p = p / np.sum(p)
    c = np.random.choice(vocab_size, 1, p=p)[0]
    return c

In [35]:
preds, vocab_size, top_n=np.array([[0.1],[0],[0.2],[0.7]]),4,2
#np.squeeze 移除长度是1的轴，例如shape=(2,1) 变为（2,）
print(preds.shape)
p = np.squeeze(preds)
print('p=' ,p)
print('p的shape',p.shape)

#argsort函数返回的是数组值从小到大的索引值
print('argsort 后的结果',np.argsort(p))

#
print(p[np.argsort(p)] )
print(p[np.argsort(p)[:-1]])
print(p[np.argsort(p)[:-2]])

p[np.argsort(p)[:-top_n]] = 0
print('设置排序后的数据的前2个 ,这里2个是因为top_n=2')
p = p / np.sum(p)
print(p)

print(np.arange(4))

#挑选1个值，从随机数中挑选一个值，挑选的值来源是 np.arange(vocab_size) ，也就是[0 1 2 3]，挑选值的概率采用后面的p参数

c = np.random.choice(vocab_size, 1, p=p)[0]
print(c)

(4, 1)
p= [ 0.1  0.   0.2  0.7]
p的shape (4,)
argsort 后的结果 [1 0 2 3]
[ 0.   0.1  0.2  0.7]
[ 0.   0.1  0.2]
[ 0.   0.1]
设置排序后的数据的前2个 ,这里2个是因为top_n=2
[ 0.          0.          0.22222222  0.77777778]
[0 1 2 3]
3


In [38]:
checkpoint = tf.train.latest_checkpoint('checkpoints')
n_samples, lstm_size,prime=5,10,"月"
samples = [c for c in prime]
print(samples)
#传入参数sampling=True 实现单个字符的连续预测
model = CharRNN(len(vocab), lstm_size=lstm_size, sampling=True)
print(model)
saver = tf.train.Saver()
with tf.Session() as sess:
    saver.restore(sess, checkpoint)
    #设置初始状态全部为0
    new_state = sess.run(model.initial_state)
    
    #对输入单词做循环例如输入"仙人",循环获得 “仙”，“人”，对应的字典编码
    for c in prime:
        print(c)
        #x 设置为 [[ 0.]]
        x = np.zeros((1, 1))
        #当c=“人”，x=[[ 164.]]，164是人的字典编码
        x[0,0] = vocab_to_int[c]
        #预测是要保证keep_prob=0
        feed = {model.inputs: x,
                model.keep_prob: 1.,
                model.initial_state: new_state}
        #预测为output函数中的：tf.nn.softmax(logits, name='predictions'),
        #例如循环到‘人’ ，从字典挑出某个词的概率。如果字典有245个文字。就有245为长度的数组，相加等于1
        
        preds, new_state = sess.run([model.prediction, model.final_state], 
                                     feed_dict=feed)
        print(preds)
        print(np.sum(preds))
        
    #获得一个单词的下标
    c = pick_top_n(preds, len(vocab))
    #把单词追加到现有单词后面
    samples.append(int_to_vocab[c])
    print(samples)

    #继续循环知道直到达到设置预测字符个数的限制
    for i in range(n_samples):
        x[0,0] = c
        feed = {model.inputs: x,
                model.keep_prob: 1.,
                model.initial_state: new_state}
        preds, new_state = sess.run([model.prediction, model.final_state], 
                                     feed_dict=feed)

        c = pick_top_n(preds, len(vocab))
        samples.append(int_to_vocab[c])
        print(samples)
    #以无分隔符''的方式把字符list合并
    print(''.join(samples))

['月']
build_inputs 函数中的input <bound method Kernel.raw_input of <ipykernel.ipkernel.IPythonKernel object at 0x0000000004BB3860>>
build LSTM函数中传入的batch_size 1
Tensor("one_hot:0", shape=(1, 1, 1334), dtype=float32)
build_output 函数中的seq_output 是否为10列 60行 ： Tensor("concat:0", shape=(1, 1, 10), dtype=float32)
build_output 函数中的 reshape seqput  Tensor("Reshape:0", shape=(1, 10), dtype=float32)
build_output 函数中的logit 是否是字典个数: Tensor("add:0", shape=(1, 1334), dtype=float32)
build_output 函数中的softmax_out Tensor("predictions:0", shape=(1, 1334), dtype=float32)
月
[[ 0.00075171  0.00075193  0.00075007 ...,  0.00075017  0.00075003
   0.00074883]]
1.0
['月', '南']
['月', '南', '入']
['月', '南', '入', '响']
['月', '南', '入', '响', '气']
['月', '南', '入', '响', '气', '态']
['月', '南', '入', '响', '气', '态', '凑']
月南入响气态凑


In [39]:
def sample(checkpoint, n_samples, lstm_size, vocab_size, prime="小"):
    
    samples = [c for c in prime]
    model = CharRNN(len(vocab), lstm_size=lstm_size, sampling=True)
    saver = tf.train.Saver()
    with tf.Session() as sess:
        saver.restore(sess, checkpoint)
        new_state = sess.run(model.initial_state)
        for c in prime:
            x = np.zeros((1, 1))
            x[0,0] = vocab_to_int[c]
            feed = {model.inputs: x,
                    model.keep_prob: 1.,
                    model.initial_state: new_state}
            preds, new_state = sess.run([model.prediction, model.final_state], 
                                         feed_dict=feed)

        c = pick_top_n(preds, len(vocab))
        samples.append(int_to_vocab[c])

        for i in range(n_samples):
            x[0,0] = c
            feed = {model.inputs: x,
                    model.keep_prob: 1.,
                    model.initial_state: new_state}
            preds, new_state = sess.run([model.prediction, model.final_state], 
                                         feed_dict=feed)

            c = pick_top_n(preds, len(vocab))
            samples.append(int_to_vocab[c])
        
    return ''.join(samples)

In [40]:
tf.train.latest_checkpoint('checkpoints')

'checkpoints\\i2340_l10.ckpt'

In [49]:
checkpoint = tf.train.latest_checkpoint('checkpoints')
samp = sample(checkpoint,3, lstm_size, len(vocab), prime="日")
print(samp)

build_inputs 函数中的input <bound method Kernel.raw_input of <ipykernel.ipkernel.IPythonKernel object at 0x0000000004BB3860>>
build LSTM函数中传入的batch_size 1
Tensor("one_hot:0", shape=(1, 1, 1334), dtype=float32)
build_output 函数中的seq_output 是否为10列 60行 ： Tensor("concat:0", shape=(1, 1, 10), dtype=float32)
build_output 函数中的 reshape seqput  Tensor("Reshape:0", shape=(1, 10), dtype=float32)
build_output 函数中的logit 是否是字典个数: Tensor("add:0", shape=(1, 1334), dtype=float32)
build_output 函数中的softmax_out Tensor("predictions:0", shape=(1, 1334), dtype=float32)
日不童俯悠
